In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import glob
import os

In [ ]:
def find_ginseng(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, w = img.shape[:2]
    src_gray = img[:,:, 1]
    bg = np.zeros((h, w, 3), np.uint8)
    ret,th1 = cv2.threshold(src_gray,50 ,255, cv2.THRESH_BINARY)
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    close = cv2.morphologyEx(th1, cv2.MORPH_CLOSE, k)
    contours, _ = cv2.findContours(
        close, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for i in range(len(contours)):
            if cv2.contourArea(contours[i]) > 30000:
                cv2.drawContours(bg, contours, i, (120,120,120), 1, cv2.LINE_AA)
                cnt = contours[i]
                x, y, w, h = cv2.boundingRect(cnt)
                cv2.rectangle(bg, (x, y), (x+w, y+h), (0, 255, 0), 3)
                cropped_img = img[y:y+h, x:x+w]
                cropped_img = cv2.cvtColor(cropped_img, cv2.COLOR_HSV2BGR)
    return cropped_img

In [ ]:
def remove_background(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    src_gray = img[:,:, 1]
    h, w = img.shape[:2]
    bg = np.zeros((h, w, 3), np.uint8)
    ret,th1 = cv2.threshold(src_gray,40 ,255, cv2.THRESH_BINARY)
    k = cv2.getStructuringElement(cv2.MORPH_RECT, (13,13))
    k2 = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    close = cv2.morphologyEx(th1, cv2.MORPH_CLOSE, k2)
    opening = cv2.morphologyEx(close, cv2.MORPH_OPEN, k)
    mask_inv = cv2.bitwise_not(opening)
    h, w = img.shape[:2]
    img1_bg = cv2.bitwise_and(bg, bg, mask=mask_inv)
    img2_fg = cv2.bitwise_and(img, img, mask=opening)
    dst = cv2.add(img1_bg, img2_fg)
    dst = cv2.cvtColor(dst, cv2.COLOR_HSV2BGR)
    return dst, opening

In [ ]:
def remove_noise(img, opening):
    img = img
    h, w = img.shape[:2]
    bg = np.zeros((h, w, 1), np.uint8)
    mask_inv = cv2.bitwise_not(opening)
    img1_bg = cv2.bitwise_and(bg, bg, mask=mask_inv)
    img2_fg = cv2.bitwise_and(img, img, mask=opening)
    dst = cv2.add(img1_bg, img2_fg)
    return dst

In [ ]:
def make_clahe(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    lab_Lightness, lab_redness, lab_Yellowness = cv2.split(img)
    clahe = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(10,10))
    Lightness = clahe.apply(lab_Lightness)
    Redness = clahe.apply(lab_redness)
    Yellowness = clahe.apply(lab_Yellowness)
    lab = cv2.merge((Lightness,Redness,Yellowness))
    clahe_lab = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    return clahe_lab

In [ ]:
def sized_img(img):
    img = img
    h, w = img.shape[:2]
    if h > w:
        bg = np.zeros((h,h,3), np.uint8)
        x_offset = (h - w) // 2
        y_offset = (h - h) // 2
        bg[:, x_offset:x_offset+w] = img
    else:
        bg = np.zeros((w,w,3), np.uint8)
        x_offset = (w - w) // 2
        y_offset = (w - h) // 2
        bg[y_offset:y_offset+h, x_offset:x_offset+w] = img
    return bg


In [ ]:
def background_size(img):
    h, w = img.shape[:2]
    if h > w:
        bg = np.zeros((h,h), np.uint8)
        x_offset = (h - w) // 2
        y_offset = (h - h) // 2
        bg[:, x_offset:x_offset+w] = img
    else:
        bg = np.zeros((w,w), np.uint8)
        x_offset = (w - w) // 2
        y_offset = (w - h) // 2
        bg[y_offset:y_offset+h, x_offset:x_offset+w] = img
    return bg

In [ ]:
Folder = glob.glob('D:/ginseng/grade/image/original/3/*.jpg')
count = 30001
for filename in Folder:
    img = cv2.imread(f'{filename}', cv2.IMREAD_COLOR)
    dst = find_ginseng(img)
    dst2, opening = remove_background(dst)
    dst3 = sized_img(dst2)
    clahe = make_clahe(dst3)
    # gray = cv2.cvtColor(clahe, cv2.COLOR_BGR2GRAY)
    # blur = cv2.GaussianBlur(dst3, (0,0), 5)
    mask_sized = background_size(opening)
    # dst4 = remove_noise(gray,mask_sized)
    resized = cv2.resize(mask_sized, (224,224), cv2.INTER_AREA)
    cv2.imwrite(f'D:/article/image/mask/3/{count}.bmp', resized)
    count += 1



In [ ]:
img = cv2.imread('D:/ginseng/grade/image/original/1/10003.jpg', cv2.IMREAD_COLOR)
dst = find_ginseng(img)
dst2, opening = remove_background(dst)
dst3 = sized_img(opening)
# clahe = make_clahe(dst3)
# gray = cv2.cvtColor(clahe, cv2.COLOR_BGR2GRAY)
# blur = cv2.GaussianBlur(dst3, (0,0), 3)
# resized = cv2.resize(clahe, (224,224))
# cv2.imshow('dst', dst3)
# cv2.imshow('dst2', resized)
# cv2.waitKey()
# cv2.destroyAllWindows()